In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [3]:
data_folder =  "../data/HWTK/en"

In [4]:
ls "../data/HWTK/en/"

hol.feather  itc_en.feather  it_en.feather  sh_en.feather


In [5]:
df_shop = pd.read_feather(os.path.join(data_folder, "sh_en.feather"))
df_item = pd.read_feather(os.path.join(data_folder, "it_en.feather"))
df_item_category = pd.read_feather(os.path.join(data_folder, "itc_en.feather"))
df_train = pd.read_csv(os.path.join("../data/sales_train_v2.csv"))
df_hol = pd.read_feather(os.path.join(data_folder, "hol.feather"))

In [6]:
df_item_category.columns = ["item_category_name", "item_category_id", "item_category_name_in_en"]

In [7]:
df_item_category["item_category_name_in_en"] = df_item_category.item_category_name_in_en.str.lower()

In [8]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
df_train["date"] = pd.to_datetime(df_train.date, format="%d.%m.%Y")

In [10]:
item_age = (df_train
            .groupby("item_id")
            .aggregate({"date" : ["first", "last"]}))
item_age.columns = item_age.columns.droplevel(0)
item_age["age"] = item_age["last"] - item_age["first"]

In [11]:
item_age.head()

,first,last,age
item_id,,,
0,2014-09-01,2014-09-01,0 days
1,2014-04-04,2014-10-20,199 days
2,2014-08-24,2014-11-12,80 days
3,2014-07-05,2014-08-26,52 days
4,2014-09-19,2014-09-19,0 days


In [12]:
df_item.columns = ["item_name", "item_id", "item_category_id", "item_english_name"]

In [13]:
df_item = df_item.merge(item_age, how="left", left_on="item_id", right_index=True)

In [14]:
df_item.head()

,item_name,item_id,item_category_id,item_english_name,first,last,age
0,ВО ВЛАСТИ НАВАЖДЕНИЯ ПЛАСТ D,0,40.0,IN THE FIRE OF DREATING THE PLAST D,2014-09-01,2014-09-01,0 days
1,ABBYY FineReader 12 Professional Edition Full ...,1,76.0,ABBYY FineReader 12 Professional Edition Full ...,2014-04-04,2014-10-20,199 days
2,В ЛУЧАХ СЛАВЫ UNV D,2,40.0,In the glories of the glory UNV D,2014-08-24,2014-11-12,80 days
3,ГОЛУБАЯ ВОЛНА Univ D,3,40.0,BLUE WAVE Univ D,2014-07-05,2014-08-26,52 days
4,КОРОБКА СТЕКЛО D,4,40.0,BOX GLASS D,2014-09-19,2014-09-19,0 days


In [15]:
df_train = (df_train
            .merge(df_item, on="item_id", how="left")
            .loc[:, ["date_block_num", "shop_id", "item_id", "first", "last", "age", "item_category_id", "item_cnt_day"]])

In [16]:
df_train.head()

,date_block_num,shop_id,item_id,first,last,age,item_category_id,item_cnt_day
0,0,59,22154,2013-01-02,2015-10-10,1011 days,37.0,1.0
1,0,25,2552,2013-01-03,2015-05-27,874 days,58.0,1.0
2,0,25,2552,2013-01-03,2015-05-27,874 days,58.0,-1.0
3,0,25,2554,2013-01-06,2013-01-06,0 days,58.0,1.0
4,0,25,2555,2013-01-15,2014-10-18,641 days,56.0,1.0


In [17]:
def clean_and_simplify_english_name(x):
    x = x.lower()
    if "accessor" in x: return "acc"
    elif "game consoles" in x: return "gc" 
    elif "games" in x: return "games"
    elif "books" in x: return "books"
    elif "programs" in x: return "programs"
    elif "gift" in x: return "gift"
    elif "movie" in x: return "movie"
    elif "music" in x: return "music" 
    else: return "misc"

In [18]:
def get_console_type(x):
    if "ps2" in x: return "ps2"
    elif "ps3" in x: return "ps3"
    elif "ps4" in x: return "ps4"
    elif "pc" in x: return "pc"
    elif "ps4" in x: return "ps4"
    elif "xbox 360" in x: return "xbox 360"
    elif "xbox one" in x: return "xbox one"
    else: return "zero"

In [19]:
df_item_category.head()

,item_category_name,item_category_id,item_category_name_in_en
0,PC Гарнитуры Наушники,0,pc headsets headphones
1,Аксессуары PS2,1,ps2 accessories
2,Аксессуары PS3,2,ps3 accessories
3,Аксессуары PS4,3,ps4 accessories
4,Аксессуары PSP,4,psp accessories


In [20]:
df_item_category["console_type"] = df_item_category.item_category_name_in_en.apply(get_console_type)
df_item_category["item_category_name_in_en"] = df_item_category.item_category_name_in_en.apply(clean_and_simplify_english_name)

In [21]:
df_item_category.head()

,item_category_name,item_category_id,item_category_name_in_en,console_type
0,PC Гарнитуры Наушники,0,misc,pc
1,Аксессуары PS2,1,acc,ps2
2,Аксессуары PS3,2,acc,ps3
3,Аксессуары PS4,3,acc,ps4
4,Аксессуары PSP,4,acc,zero


In [22]:
df_train = df_train.merge(df_item_category, on="item_category_id", how="left")

In [23]:
df_train.head()

,date_block_num,shop_id,item_id,first,last,age,item_category_id,item_cnt_day,item_category_name,item_category_name_in_en,console_type
0,0,59,22154,2013-01-02,2015-10-10,1011 days,37,1.0,Кино Blu Ray,movie,zero
1,0,25,2552,2013-01-03,2015-05-27,874 days,58,1.0,Музыка Винил,music,zero
2,0,25,2552,2013-01-03,2015-05-27,874 days,58,-1.0,Музыка Винил,music,zero
3,0,25,2554,2013-01-06,2013-01-06,0 days,58,1.0,Музыка Винил,music,zero
4,0,25,2555,2013-01-15,2014-10-18,641 days,56,1.0,Музыка CD фирменного производства,music,zero


In [24]:
df_train["week_age"] = df_train.age.dt.days / 7.0

In [25]:
df_train.head()

,date_block_num,shop_id,item_id,first,last,age,item_category_id,item_cnt_day,item_category_name,item_category_name_in_en,console_type,week_age
0,0,59,22154,2013-01-02,2015-10-10,1011 days,37,1.0,Кино Blu Ray,movie,zero,144.428571
1,0,25,2552,2013-01-03,2015-05-27,874 days,58,1.0,Музыка Винил,music,zero,124.857143
2,0,25,2552,2013-01-03,2015-05-27,874 days,58,-1.0,Музыка Винил,music,zero,124.857143
3,0,25,2554,2013-01-06,2013-01-06,0 days,58,1.0,Музыка Винил,music,zero,0.000000
4,0,25,2555,2013-01-15,2014-10-18,641 days,56,1.0,Музыка CD фирменного производства,music,zero,91.571429


In [26]:
df = df_train.groupby(["date_block_num", "shop_id", "item_id"]).aggregate({"item_cnt_day": np.sum, "week_age": np.mean})

In [28]:
df.sample(20)

,,,week_age,item_cnt_day
date_block_num,shop_id,item_id,,
7,31,9615,68.857143,1.0
11,26,17763,33.285714,1.0
20,27,9355,57.000000,6.0
0,6,4688,58.857143,1.0
4,47,15004,90.428571,1.0
22,54,20532,78.714286,1.0
9,22,7199,94.428571,1.0
7,28,15455,142.571429,3.0
11,6,4845,68.714286,6.0


In [105]:
tr %<>% 
  left_join(itc_en %>% select(itc_id, mtype), by='itc_id')

tr %<>% 
  mutate(week_age = as.numeric(difftime(date, age, units="weeks"))) %>% 
  select(-recency)
# Main data frame define
df <- tr %>% 
  group_by(dbn, s_id, it_id) %>% 
  summarise(sales = sum(count), avg_age = mean(week_age)) %>% 
  ungroup

In [106]:
head(df)

dbn,s_id,it_id,sales,avg_age
0,0,32,6,2.714286
0,0,33,3,2.142857
0,0,35,1,1.142857
0,0,43,1,3.571429
0,0,51,2,2.857143
0,0,61,1,0.000000


In [107]:
# Add previous month for joining with other df
df %<>% 
  mutate(prev_dbn = dbn-1)

# previous month sales average for a shop and total number 
# of items each shop selling
df %<>% 
  left_join(df %>% 
              group_by(dbn, s_id) %>% 
              summarize(shop_sales = mean(sales), it_count = n()),
            by=c("prev_dbn" = "dbn", "s_id"))

# previous month item average sales and total number of shop's selling them.
df %<>% 
left_join(df %>% 
            group_by(dbn, it_id) %>% 
            summarize(it_sales = mean(sales), shop_count = n()),
          by=c("prev_dbn" = "dbn", "it_id"))

# previous month shop, item sales
df %<>% 
  left_join(df %>% 
              select(dbn, s_id, it_id, sales) %>% 
              rename(prev_sales = sales),
            by = c("prev_dbn" = "dbn", "s_id", "it_id")) 


In [108]:
df %>% sample_n(20)

dbn,s_id,it_id,sales,avg_age,prev_dbn,shop_sales,it_count,it_sales,shop_count,prev_sales
18,56,2578,2,3.142857,17,1.787543,1445,2.727273,22,NA
6,47,14931,2,29.214286,5,1.841871,898,3.310345,29,2
4,51,18509,3,18.904762,3,1.651104,1178,1.285714,7,3
1,31,16229,5,5.642857,0,3.004430,3386,1.571429,7,1
20,31,11903,1,34.000000,19,2.943612,2802,1.375000,8,3
14,4,2854,4,62.678571,13,1.896350,685,2.096774,31,1
2,19,6340,1,8.428571,1,2.067104,1371,1.000000,1,NA
1,15,3077,1,7.714286,0,2.214634,1230,2.750000,28,3
32,42,21672,1,39.428571,31,2.448562,1565,1.100000,10,2
23,19,10748,2,1.142857,22,2.155242,992,NA,NA,NA


In [109]:
# Add lag features
lag_times = c(2, 3 ,6, 9, 12)

for (i in lag_times){
    lag_field  <- paste('lag_sales_', as.character(i), sep = '')
    
    df %<>% 
        mutate(lag_dbn = dbn-i) %>% 
        left_join(df %>% 
                    select(dbn, s_id, it_id, sales) %>% 
                    rename(!!lag_field := sales),
                  by = c("lag_dbn" = "dbn", "s_id", "it_id"))
}

In [141]:
df %>% sample_n(10)

dbn,s_id,it_id,sales,avg_age,prev_dbn,shop_sales,it_count,it_sales,shop_count,prev_sales,lag_dbn,lag_sales_2,lag_sales_3,lag_sales_6,lag_sales_9,lag_sales_12
28,31,21619,1,124.571429,27,3.060025,2399,1.250000,8,1,16,1,4,NA,4,1
30,47,13300,2,1.500000,29,2.148148,810,NA,NA,NA,18,NA,NA,NA,NA,NA
28,25,6625,2,56.357143,27,2.864146,2142,1.083333,12,1,16,1,1,1,NA,2
24,21,4334,1,9.857143,23,2.539448,1559,4.279070,43,2,12,1,NA,NA,NA,NA
11,31,7055,3,14.190476,10,3.504939,3341,1.733333,15,2,-1,5,14,NA,NA,NA
22,56,17890,2,24.285714,21,1.696123,1109,1.000000,5,NA,10,NA,2,2,NA,NA
20,28,7213,1,77.571429,19,3.263217,1778,1.724138,29,2,8,2,3,4,6,1
27,48,7895,2,119.142857,26,1.544855,758,2.047619,21,1,15,NA,2,1,1,NA
7,6,6635,1,32.285714,6,1.906924,1762,1.214286,14,NA,-5,NA,NA,NA,NA,NA
29,59,15063,1,109.428571,28,1.810277,506,1.428571,21,1,17,NA,2,2,NA,1


In [125]:
# Replace all na with 0

sales_columns <- str_detect(colnames(df), 'sales')

In [132]:
df[,sales_columns] %>% replace_na(0)

ERROR: Error: is_list(replace) is not TRUE


In [137]:
df[,sales_columns]  %>% replace_na(rep(0, nrow(.))) %>% head

ERROR: Error: is_list(replace) is not TRUE


In [139]:
seq(0, nrow(df))